<a href="https://colab.research.google.com/github/aai540-group3/project/blob/main/aws_confgure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configure AWS

## Install Dependencies

In [ ]:
%%bash

# INSTALL: uv
# Rust-based Python Package Management
pip install --quiet --progress-bar=off uv

# INSTALL: boto3
# AWS SDK (Software Development Kit) for Python
uv pip install --system --quiet boto3

mkdir -p /root/.aws

## Add Credentials to Google Colab

In [52]:
from google.colab import userdata
from google.colab.userdata import SecretNotFoundError

def get_aws_credential(credential_name):
    """
    Retrieve an AWS credential from Google Colab Secrets.

    :param credential_name: The name of the credential to retrieve
    :type credential_name: str
    :return: The retrieved credential value
    :rtype: str
    :raises SecretNotFoundError: If the specified credential is not found in Colab Secrets
    """
    try:
        return userdata.get(credential_name)
    except SecretNotFoundError as e:
        print(f"Error: '{credential_name}' not found.")
        print_secret_setup_instructions(credential_name)
        raise

def print_secret_setup_instructions(secret_name):
    """
    Print instructions for setting up a secret in Google Colab Secrets UI.

    :param secret_name: The name of the secret to set up
    :type secret_name: str
    """
    print(f"To set it manually in the Google Colab Secrets UI:")
    print("1. Click the key icon on the left sidebar to open the 'Secrets' pane.")
    print("2. Click the '+' button to add a new secret.")
    print(f"3. Enter '{secret_name}' as the name and your actual AWS credential as the value.")
    print("4. Save the secret and rerun the code cell.")

def main():
    """
    Main function to retrieve AWS credentials from Google Colab Secrets.

    :return: A dictionary indicating whether each credential was successfully loaded
    :rtype: dict
    """
    credentials_status = {}
    credential_names = ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY']

    for credential_name in credential_names:
        try:
            get_aws_credential(credential_name)
            credentials_status[credential_name] = True
            print(f"{credential_name}: LOADED SECRET")
        except SecretNotFoundError:
            credentials_status[credential_name] = False
            print(f"{credential_name}: FAILED TO LOAD SECRET")

    return credentials_status

if __name__ == "__main__":
    main()

AWS_ACCESS_KEY_ID: LOADED SECRET
AWS_SECRET_ACCESS_KEY: LOADED SECRET


## Create Credential And Config Files

In [53]:
import os
import textwrap

def write_aws_credentials(aws_access_key_id, aws_secret_access_key, file_path="/root/.aws/credentials"):
    """
    Write AWS credentials to a file.

    :param aws_access_key_id: The AWS access key ID
    :type aws_access_key_id: str
    :param aws_secret_access_key: The AWS secret access key
    :type aws_secret_access_key: str
    :param file_path: The path to the credentials file (default: "/root/.aws/credentials")
    :type file_path: str
    """
    credentials_content = textwrap.dedent(f"""
    [default]
    aws_access_key_id = {aws_access_key_id}
    aws_secret_access_key = {aws_secret_access_key}
    """).strip()
    write_file(file_path, credentials_content)

def write_aws_config(region="us-west-2", file_path="/root/.aws/config"):
    """
    Write AWS config to a file.

    :param region: The AWS region (default: "us-west-2")
    :type region: str
    :param file_path: The path to the config file (default: "/root/.aws/config")
    :type file_path: str
    """
    config_content = textwrap.dedent(f"""
    [default]
    region = {region}
    """).strip()
    write_file(file_path, config_content)

def write_file(file_path, content):
    """
    Write content to a file, creating the directory if it doesn't exist.

    :param file_path: The path to the file
    :type file_path: str
    :param content: The content to write to the file
    :type content: str
    """
    directory = os.path.dirname(file_path)
    os.makedirs(directory, exist_ok=True)

    with open(file_path, "w") as f:
        f.write(content)

def setup_aws_credentials_and_config(aws_access_key_id, aws_secret_access_key, region="us-west-2"):
    """
    Set up AWS credentials and config files.

    :param aws_access_key_id: The AWS access key ID
    :type aws_access_key_id: str
    :param aws_secret_access_key: The AWS secret access key
    :type aws_secret_access_key: str
    :param region: The AWS region (default: "us-west-2")
    :type region: str
    """
    write_aws_credentials(aws_access_key_id, aws_secret_access_key)
    write_aws_config(region)
    print("AWS credentials and config files have been created successfully.")

if __name__ == "__main__":
    setup_aws_credentials_and_config(aws_access_key_id, aws_secret_access_key)

AWS credentials and config files have been created successfully.


## Test Credentials

In [57]:
import boto3
from botocore.exceptions import BotoCoreError, ClientError

def create_aws_session(profile_name='default'):
    """
    Create an AWS session using the specified profile.

    :param profile_name: The name of the AWS profile to use (default: 'default')
    :type profile_name: str
    :return: An AWS session object
    :rtype: boto3.Session
    """
    return boto3.Session(profile_name=profile_name)

def get_aws_identity(session):
    """
    Retrieve the AWS identity information using the provided session.

    :param session: The AWS session to use
    :type session: boto3.Session
    :return: The response from the get_caller_identity API call
    :rtype: dict
    :raises BotoCoreError: If there's an error in the boto3 core
    :raises ClientError: If there's an error in the client's request
    """
    sts_client = session.client('sts')
    return sts_client.get_caller_identity()

def check_aws_authentication(profile_name='default'):
    """
    Check AWS authentication by attempting to retrieve the caller's identity.

    :param profile_name: The name of the AWS profile to use (default: 'default')
    :type profile_name: str
    :return: A boolean indicating whether authentication was successful
    :rtype: bool
    """
    try:
        session = create_aws_session(profile_name)
        get_aws_identity(session)
        return True
    except (BotoCoreError, ClientError):
        return False

def main():
    """
    Main function to check AWS authentication and print the status.
    """
    is_authenticated = check_aws_authentication()
    status = "AUTHENTICATED" if is_authenticated else "NOT AUTHENTICATED"
    print(f"STATUS: {status}")

if __name__ == "__main__":
    main()

STATUS: AUTHENTICATED


## Add Users to Organization

In [43]:
import boto3
import time

def check_account_exists(email):
    """
    Check if an AWS account with the given email already exists.

    :param email: The email address to check
    :type email: str
    :return: True if the account exists, False otherwise
    :rtype: bool
    """
    org_client = boto3.client('organizations')
    paginator = org_client.get_paginator('list_accounts')

    for page in paginator.paginate():
        for account in page['Accounts']:
            if account['Email'] == email:
                return True
    return False

def create_account(email):
    """
    Create a new AWS account with the given email.

    :param email: The email address for the new account
    :type email: str
    :return: The create account status ID if successful, None otherwise
    :rtype: str or None
    """
    org_client = boto3.client('organizations')
    try:
        response = org_client.create_account(
            Email=email,
            AccountName=email,
            RoleName='OrganizationAccountAccessRole'
        )
        return response['CreateAccountStatus']['Id']
    except Exception as e:
        print(f"Error creating account for {email}: {e}")
        return None

def track_account_creation(create_status_id, email):
    """
    Track the status of an account creation request.

    :param create_status_id: The ID of the create account request
    :type create_status_id: str
    :param email: The email address of the account being created
    :type email: str
    :return: None
    """
    org_client = boto3.client('organizations')
    while True:
        try:
            status_response = org_client.describe_create_account_status(
                CreateAccountRequestId=create_status_id
            )
            status = status_response['CreateAccountStatus']['State']
            if status == 'SUCCEEDED':
                print(f"SUCCEEDED: {status_response['CreateAccountStatus']['AccountId']}")
                break
            elif status == 'FAILED':
                print(f"FAILED: {status_response['CreateAccountStatus']['FailureReason']}")
                break
            else:
                print(f"CREATING: {email}")
                print(f"    STATUS: {status}")
                time.sleep(10)
        except Exception as e:
            print(f"Error tracking account creation: {e}")
            break

def main():
    """
    Main function to create AWS accounts for a list of email addresses.
    """
    emails_to_create = [
        'jagustin@sandiego.edu',
        'lvo@sandiego.edu',
        'zrobertson@sandiego.edu'
    ]

    for email in emails_to_create:
        if check_account_exists(email):
            print(f"EXISTS: {email}")
        else:
            create_status_id = create_account(email)
            if create_status_id:
                print(f"CREATING: {email}")
                track_account_creation(create_status_id, email)

if __name__ == "__main__":
    main()

EXISTS: jagustin@sandiego.edu
EXISTS: lvo@sandiego.edu
EXISTS: zrobertson@sandiego.edu
